In [2]:
!pip install -Uqqq pip
!pip install -qqq langchain==0.1.5
!pip install -qqq openai==1.11.1
!pip install -qqq watermark==2.4.3
!pip install -Uqqq chromadb==0.4.22 
!pip install -qqq tiktoken==0.5.2 
!pip install -qqq langchain-openai==0.0.5

In [8]:
import os
import textwrap
from getpass import getpass

import chromadb
import langchain
import openai

# from langchain_experimental.llm_bash.base import LLMBashChain
from langchain.chains import LLMChain, RetrievalQA, SimpleSequentialChain
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI

from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from dotenv import load_dotenv

In [6]:
%load_ext watermark
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.11.5
IPython version      : 8.21.0

Compiler    : Clang 14.0.0 (clang-1400.0.29.202)
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

chromadb : 0.4.22
openai   : 1.11.1
langchain: 0.1.5



In [7]:
def print_response(response: str):
    print("\n".join(textwrap.wrap(response, width=100)))

In [9]:
# OPENAI_API_KEY = getpass()
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
load_dotenv()

True

In [10]:

chat_gpt = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [11]:
template = """
You have to come up with location to shoot (along with a 20-50 word description)
for a new episode of the TV show "The Office" based on the theme

{theme_suggestion}

ANSWER:
"""
prompt = PromptTemplate(input_variables=["theme_suggestion"], template=template)

location_chain = LLMChain(llm=chat_gpt, prompt=prompt)

In [13]:
response = location_chain.invoke("Visiting Asia")
response

{'theme_suggestion': 'Visiting Asia',
 'text': 'Title: "The Office: Asian Adventure"\n\nDescription: In this special episode, the Dunder Mifflin Scranton branch embarks on an unforgettable journey to Asia. Join Michael Scott and his quirky team as they navigate the bustling streets of Tokyo, explore ancient temples in Kyoto, and attempt to master the art of karaoke in Seoul. Hilarity ensues as cultural misunderstandings and misadventures abound, making this Asian adventure one for the books.'}

In [14]:
print_response(response["text"])

Title: "The Office: Asian Adventure"  Description: In this special episode, the Dunder Mifflin
Scranton branch embarks on an unforgettable journey to Asia. Join Michael Scott and his quirky team
as they navigate the bustling streets of Tokyo, explore ancient temples in Kyoto, and attempt to
master the art of karaoke in Seoul. Hilarity ensues as cultural misunderstandings and misadventures
abound, making this Asian adventure one for the books.


## SequentialChain

In [15]:
template = """
Generate a short dialogue between Jim and Pam
from the TV show "The Office" for a new episode based on the location

{location}

ANSWER:
"""
prompt = PromptTemplate(input_variables=["location"], template=template)

conversation_chain = LLMChain(llm=chat_gpt, prompt=prompt)

In [17]:
sequential_chain = SimpleSequentialChain(
    chains=[location_chain, conversation_chain], verbose=True
)
response = sequential_chain.invoke("Visiting Europe")



> Entering new SimpleSequentialChain chain...
Title: "The Office: European Escapades"

Description: The Dunder Mifflin Scranton branch embarks on a hilarious European adventure! Join Michael Scott and his quirky team as they navigate cultural differences, language barriers, and questionable fashion choices. From the bustling streets of London to the romantic canals of Venice, this episode promises laughter, misunderstandings, and a whole lot of cringe-worthy moments.
INT. SCRANTON BRANCH - DUNDER MIFFLIN - DAY

Jim and Pam are sitting at their desks, looking excited and slightly nervous.

JIM
(whispering)
Can you believe it, Pam? We're actually going to Europe with the whole office!

PAM
(smiling)
I know, right? It's going to be so much fun! I can't wait to see Michael's reaction to all the cultural differences.

INT. AIRPLANE - DAY

Jim and Pam are seated next to each other on the plane, wearing matching neck pillows.

JIM
(looking out the window)
Hey, Pam, look at that view! We're 

In [18]:
conversation = response
print(conversation)

{'input': 'Visiting Europe', 'output': 'INT. SCRANTON BRANCH - DUNDER MIFFLIN - DAY\n\nJim and Pam are sitting at their desks, looking excited and slightly nervous.\n\nJIM\n(whispering)\nCan you believe it, Pam? We\'re actually going to Europe with the whole office!\n\nPAM\n(smiling)\nI know, right? It\'s going to be so much fun! I can\'t wait to see Michael\'s reaction to all the cultural differences.\n\nINT. AIRPLANE - DAY\n\nJim and Pam are seated next to each other on the plane, wearing matching neck pillows.\n\nJIM\n(looking out the window)\nHey, Pam, look at that view! We\'re flying over the English Channel.\n\nPAM\n(excitedly)\nI know, it\'s amazing! I can\'t wait to explore London. I\'ve always wanted to see Big Ben and the Tower Bridge.\n\nINT. LONDON - DAY\n\nJim and Pam are walking through the busy streets of London, holding a map and looking lost.\n\nJIM\n(confused)\nPam, I think we took a wrong turn. This map is so confusing.\n\nPAM\n(giggling)\nYeah, I think we\'re defini

## Summarization

In [19]:
template = """
Write a concise bullet list summary of the conversation between Jim and Pam from the TV show "The Office":

{text}

Concise summary using markdown:"""

prompt = PromptTemplate(template=template, input_variables=["text"])
summary_chain = load_summarize_chain(
    chat_gpt, chain_type="stuff", verbose=True, prompt=prompt
)

In [21]:
conversation

{'input': 'Visiting Europe',
 'output': 'INT. SCRANTON BRANCH - DUNDER MIFFLIN - DAY\n\nJim and Pam are sitting at their desks, looking excited and slightly nervous.\n\nJIM\n(whispering)\nCan you believe it, Pam? We\'re actually going to Europe with the whole office!\n\nPAM\n(smiling)\nI know, right? It\'s going to be so much fun! I can\'t wait to see Michael\'s reaction to all the cultural differences.\n\nINT. AIRPLANE - DAY\n\nJim and Pam are seated next to each other on the plane, wearing matching neck pillows.\n\nJIM\n(looking out the window)\nHey, Pam, look at that view! We\'re flying over the English Channel.\n\nPAM\n(excitedly)\nI know, it\'s amazing! I can\'t wait to explore London. I\'ve always wanted to see Big Ben and the Tower Bridge.\n\nINT. LONDON - DAY\n\nJim and Pam are walking through the busy streets of London, holding a map and looking lost.\n\nJIM\n(confused)\nPam, I think we took a wrong turn. This map is so confusing.\n\nPAM\n(giggling)\nYeah, I think we\'re defin

In [22]:
docs = [Document(page_content=conversation['output'])]
docs

[Document(page_content='INT. SCRANTON BRANCH - DUNDER MIFFLIN - DAY\n\nJim and Pam are sitting at their desks, looking excited and slightly nervous.\n\nJIM\n(whispering)\nCan you believe it, Pam? We\'re actually going to Europe with the whole office!\n\nPAM\n(smiling)\nI know, right? It\'s going to be so much fun! I can\'t wait to see Michael\'s reaction to all the cultural differences.\n\nINT. AIRPLANE - DAY\n\nJim and Pam are seated next to each other on the plane, wearing matching neck pillows.\n\nJIM\n(looking out the window)\nHey, Pam, look at that view! We\'re flying over the English Channel.\n\nPAM\n(excitedly)\nI know, it\'s amazing! I can\'t wait to explore London. I\'ve always wanted to see Big Ben and the Tower Bridge.\n\nINT. LONDON - DAY\n\nJim and Pam are walking through the busy streets of London, holding a map and looking lost.\n\nJIM\n(confused)\nPam, I think we took a wrong turn. This map is so confusing.\n\nPAM\n(giggling)\nYeah, I think we\'re definitely lost. But h

In [23]:
summary_result = summary_chain.run(docs)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise bullet list summary of the conversation between Jim and Pam from the TV show "The Office":

INT. SCRANTON BRANCH - DUNDER MIFFLIN - DAY

Jim and Pam are sitting at their desks, looking excited and slightly nervous.

JIM
(whispering)
Can you believe it, Pam? We're actually going to Europe with the whole office!

PAM
(smiling)
I know, right? It's going to be so much fun! I can't wait to see Michael's reaction to all the cultural differences.

INT. AIRPLANE - DAY

Jim and Pam are seated next to each other on the plane, wearing matching neck pillows.

JIM
(looking out the window)
Hey, Pam, look at that view! We're flying over the English Channel.

PAM
(excitedly)
I know, it's amazing! I can't wait to explore London. I've always wanted to see Big Ben and the Tower Bridge.

INT. LONDON - DAY

Jim and Pam are walking through the busy streets of London, holding a map and l

In [24]:
print(summary_result)

- Jim and Pam are excited about going to Europe with the office.
- They discuss their anticipation of seeing Michael's reaction to cultural differences.
- They admire the view of the English Channel from the airplane.
- They express excitement about exploring London and seeing landmarks like Big Ben and the Tower Bridge.
- They get lost in London but find humor in being lost together.
- They enjoy a romantic gondola ride in Venice and joke about recreating a scene from "The Italian Job."
- They take a selfie in front of the Eiffel Tower in Paris and reflect on the amazing experience of being in Europe with their office family.


## Question Answering

In [25]:
template = """
You have to come up with a 200-300 word script for a new episode 
of the TV show "The Office" based on the theme

{theme_suggestion}

ANSWER:
"""
prompt = PromptTemplate(input_variables=["theme_suggestion"], template=template)

script_chain = LLMChain(llm=chat_gpt, prompt=prompt)

In [26]:
script_response = script_chain("Bohemian Rapsody")

In [27]:
script = script_response["text"]
print(script)

Title: "The Office: Bohemian Rhapsody"

INT. DUNDER MIFFLIN SCRANTON OFFICE - DAY

The office is buzzing with excitement as the employees prepare for the annual talent show. MICHAEL SCOTT, the regional manager, is in his office, practicing his guitar.

MICHAEL
(singing)
Is this the real life? Is this just fantasy?

JIM HALPERT, the prankster, enters the office and smirks at Michael's performance.

JIM
(smirking)
Hey, Michael. Getting ready for the talent show?

MICHAEL
(excitedly)
Yes, Jim! I'm going to perform "Bohemian Rhapsody" by Queen. It's going to be legendary!

INT. BREAK ROOM - DAY

PAM BEESLY, the receptionist, is talking to DWIGHT SCHRUTE, the eccentric assistant regional manager.

PAM
So, Dwight, are you participating in the talent show?

DWIGHT
Absolutely, Pam. I am going to showcase my impeccable karate skills. I will be the ultimate warrior!

PAM
(smiling)
That sounds interesting, Dwight. Can't wait to see it.

INT. CONFERENCE ROOM - DAY

The employees gather for the tal

In [28]:
script_docs = [Document(page_content=script)]

text_splitter = CharacterTextSplitter(chunk_size=2048, chunk_overlap=32)
texts = text_splitter.split_documents(script_docs)
len(texts)

2

In [29]:
embeddings = OpenAIEmbeddings()

In [30]:
db = Chroma.from_documents(texts, embeddings)

In [31]:
qa_chain = RetrievalQA.from_chain_type(
    llm=chat_gpt, chain_type="stuff", retriever=db.as_retriever(search_kwargs={"k": 2})
)

In [32]:
response = qa_chain("What is the location that The Office team is visiting?")
response

{'query': 'What is the location that The Office team is visiting?',
 'result': 'The given context does not provide any information about The Office team visiting a specific location.'}

In [33]:
print_response(response["result"])

The given context does not provide any information about The Office team visiting a specific
location.


In [34]:
response = qa_chain("What was the highlight of the talent show?")
print_response(response["result"])

The highlight of the talent show was Dwight's unexpected karate performance.


## Bash Chain

In [108]:
text = """
Please write a bash script that prints a single line that Michael G. Scott from "The Office" might say
"""

# bash_chain = LLMBashChain(llm=OpenAI(temperature=0), verbose=True)

# bash_chain.run(text)



> Entering new LLMBashChain chain...

Please write a bash script that prints a single line that Michael G. Scott from "The Office" might say

```bash
echo "That's what she said!"
```['```bash', 'echo "That\'s what she said!"', '```']

Answer: That's what she said!

> Finished chain.


"That's what she said!\n"